<a href="https://colab.research.google.com/github/2303a51083/AIML-2025BT17-18/blob/main/week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from queue import Queue
graph = {0: [1, 2, 3],
 1: [0, 2],
 2: [1, 3, 5],
 3: [0, 2, 4],
 4: [3, 5, 7],
 5: [2, 4, 6, 7],
 6: [5, 7],
 7:[]}
print("The adjacency List representing the graph is:")
print(graph)
def bfs(graph, source):
 Q = Queue()
 visited_vertices = set()
 Q.put(source)
 visited_vertices.update({source})
 while not Q.empty():
  vertex = Q.get()
 print(vertex, end="-->")
 for u in graph[vertex]:
  if u not in visited_vertices:
    Q.put(u)
 visited_vertices.update({u})
print("BFS traversal of graph with source 1 is:")
bfs(graph, 1)





The adjacency List representing the graph is:
{0: [1, 2, 3], 1: [0, 2], 2: [1, 3, 5], 3: [0, 2, 4], 4: [3, 5, 7], 5: [2, 4, 6, 7], 6: [5, 7], 7: []}
BFS traversal of graph with source 1 is:
1-->

In [7]:
from queue import Queue
from copy import deepcopy
import numpy as np
import time

# BFS Implementation
def bfs(graph, source):
    Q = Queue()
    visited_vertices = set()
    Q.put(source)
    visited_vertices.add(source)

    while not Q.empty():
        vertex = Q.get()
        print(vertex, end="-->")
        for u in graph[vertex]:
            if u not in visited_vertices:
                Q.put(u)
                visited_vertices.add(u)

# DFS Implementation
def dfs(graph, node, visited):
    if node not in visited:
        visited.append(node)
        for k in graph[node]:
            dfs(graph, k, visited)
    return visited

# Puzzle Solving Functions
def bestsolution(state):
    bestsol = np.array([], int).reshape(-1, 9)
    count = len(state) - 1
    while count != -1:
        bestsol = np.insert(bestsol, 0, state[count]['puzzle'], 0)
        count = state[count]['parent']
    return bestsol.reshape(-1, 3, 3)

def misplaced_tiles(puzzle, goal):
    mscost = np.sum(puzzle != goal) - 1
    return mscost if mscost > 0 else 0

def coordinates(puzzle):
    pos = np.array(range(9))
    for p, q in enumerate(puzzle):
        pos[q] = p
    return pos

def evaluvate_misplaced(puzzle, goal):
    steps = np.array([
        ('up', [0, 1, 2], -3),
        ('down', [6, 7, 8], 3),
        ('left', [0, 3, 6], -1),
        ('right', [2, 5, 8], 1)
    ], dtype=[('move', str, 1), ('position', list), ('head', int)])

    dtstate = [('puzzle', list), ('parent', int), ('gn', int), ('hn', int)]
    costg = coordinates(goal)
    parent = -1
    gn = 0
    hn = misplaced_tiles(coordinates(puzzle), costg)
    state = np.array([(puzzle, parent, gn, hn)], dtstate)

    dtpriority = [('position', int), ('fn', int)]
    priority = np.array([(0, hn)], dtpriority)

    while True:
        priority = np.sort(priority, kind='mergesort', order=['fn', 'position'])
        position, fn = priority[0]
        priority = np.delete(priority, 0, 0)
        puzzle, parent, gn, hn = state[position]
        puzzle = np.array(puzzle)
        blank = int(np.where(puzzle == 0)[0][0])  # Corrected line to avoid warning
        gn += 1

        start_time = time.time()

        for s in steps:
            if blank not in s['position']:
                openstates = deepcopy(puzzle)
                openstates[blank], openstates[blank + s['head']] = openstates[blank + s['head']], openstates[blank]

                if not np.any(np.all(list(state['puzzle']) == openstates, 1)):
                    end_time = time.time()
                    if (end_time - start_time) > 2:
                        print("The 8 puzzle is unsolvable \n")
                        break

                    hn = misplaced_tiles(coordinates(openstates), costg)
                    q = np.array([(openstates, position, gn, hn)], dtstate)
                    state = np.append(state, q, 0)
                    fn = gn + hn
                    q = np.array([(len(state) - 1, fn)], dtpriority)
                    priority = np.append(priority, q, 0)

                    if np.array_equal(openstates, goal):
                        print('The 8 puzzle is solvable \n')
                        return state, len(priority)

    return state, len(priority)

# Graphs for BFS and DFS
graph = {
    0: [1, 2, 3],
    1: [0, 2],
    2: [1, 3, 5],
    3: [0, 2, 4],
    4: [3, 5, 7],
    5: [2, 4, 6, 7],
    6: [5, 7],
    7: []
}

graph1 = {
    'A': ['B', 'C', 'G'],
    'B': ['A'],
    'C': ['A', 'D', 'F'],
    'D': ['A', 'C', 'E', 'H'],
    'E': ['D', 'H', 'F'],
    'F': ['C', 'E'],
    'G': ['A', 'D', 'H'],
    'H': ['D', 'E', 'G']
}

# Test BFS and DFS
print("BFS traversal of graph with source 1 is:")
bfs(graph, 1)

visited = dfs(graph1, 'B', [])
print("\nDFS traversal of graph starting from 'B':")
print(visited)

# Test Puzzle Solver
puzzle = [2, 8, 3, 1, 6, 4, 7, 0, 5]
goal = [1, 2, 3, 8, 0, 4, 7, 6, 5]

state, visited = evaluvate_misplaced(puzzle, goal)
bestpath = bestsolution(state)
print("\nBest solution path to solve the puzzle:")
print(str(bestpath).replace('[', ' ').replace(']', ''))
totalmoves = len(bestpath) - 1
print('\nSteps to reach goal:', totalmoves)
visit = len(state) - visited
print('Total nodes visited: ', visit, "\n")


BFS traversal of graph with source 1 is:
1-->0-->2-->3-->5-->4-->6-->7-->
DFS traversal of graph starting from 'B':
['B', 'A', 'C', 'D', 'E', 'H', 'G', 'F']
The 8 puzzle is solvable 


Best solution path to solve the puzzle:
   2 8 3
   1 6 4
   7 0 5

   2 8 3
   1 0 4
   7 6 5

   2 0 3
   1 8 4
   7 6 5

   0 2 3
   1 8 4
   7 6 5

   1 2 3
   0 8 4
   7 6 5

   1 2 3
   8 0 4
   7 6 5

Steps to reach goal: 5
Total nodes visited:  6 



In [ ]:
#CODE FOR DFS:
graph1 = {
 'A' : ['B','C','G'],
 'B' : ['A'],
 'C' : ['A','D','F',],
 'D' : ['A','C','E','H'],
 'E' : ['D','H','F'],
 'F' : ['C','E'],
 'G' : ['A','D','H'],
 'H' : ['D','E','G']
}
def dfs(graph, node, visited):
 if node not in visited:
  visited.append(node)
 for k in graph[node]:
  dfs(graph,k, visited)
 return visited
 visited = dfs(graph1,'B', [])
 print(visited)